# Multiple Input Series

In [1]:
import numpy as np
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers.merge import concatenate

Using TensorFlow backend.


In [2]:
def split_sequences(sequences, n_steps):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the dataset
		if end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
		X.append(seq_x)
		y.append(seq_y)
	return np.array(X), np.array(y)

There is another more elaborate way to model the problem. Each input series can be handled by
a separate MLP and the output of each of these submodels can be combined before a prediction
is made for the output sequence. We can refer to this as a **multi-headed input MLP model**. 

It
may offer more 
exibility or better performance depending on the specics of the problem that
are being modeled.

## 1. Define input sequence

In [3]:
in_seq1 = np.array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = np.array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = np.array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])

In [4]:
out_seq

array([ 25,  45,  65,  85, 105, 125, 145, 165, 185])

#### convert to [rows, columns] structure

In [5]:
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))

#### horizontally stack columns

In [6]:
dataset = np.hstack((in_seq1, in_seq2, out_seq))
print(dataset)

[[ 10  15  25]
 [ 20  25  45]
 [ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]
 [ 80  85 165]
 [ 90  95 185]]


## 2. Convert into input/output

In [7]:
n_steps = 3

In [8]:
X, y = split_sequences(dataset, n_steps)
print(X.shape, y.shape)

(7, 3, 2) (7,)


In [9]:
for i in range(len(X)):
    print(X[i], y[i])

[[10 15]
 [20 25]
 [30 35]] 65
[[20 25]
 [30 35]
 [40 45]] 85
[[30 35]
 [40 45]
 [50 55]] 105
[[40 45]
 [50 55]
 [60 65]] 125
[[50 55]
 [60 65]
 [70 75]] 145
[[60 65]
 [70 75]
 [80 85]] 165
[[70 75]
 [80 85]
 [90 95]] 185


## 3. Separate input data

In [10]:
X1 = X[:, :, 0]
X2 = X[:, :, 1]

In [11]:
X1

array([[10, 20, 30],
       [20, 30, 40],
       [30, 40, 50],
       [40, 50, 60],
       [50, 60, 70],
       [60, 70, 80],
       [70, 80, 90]])

In [12]:
X2

array([[15, 25, 35],
       [25, 35, 45],
       [35, 45, 55],
       [45, 55, 65],
       [55, 65, 75],
       [65, 75, 85],
       [75, 85, 95]])

## 3. Define model

### 3.1 First input model

In [13]:
visible1 = Input(shape=(n_steps,))
dense1 = Dense(100, activation='relu')(visible1)

### 3.2 Second input model

In [14]:
visible2 = Input(shape=(n_steps,))
dense2 = Dense(100, activation='relu')(visible2)

### 3.3 Merge input models

In [15]:
merge = concatenate([dense1, dense2])
output = Dense(1)(merge)
model = Model(inputs=[visible1, visible2], outputs=output)
model.compile(optimizer='adam', loss='mse')

## 5. Fit model

In [16]:
model.fit([X1, X2], y, epochs=2000, verbose=0)

## 6. Demonstrate prediction

In [17]:
x_input = np.array([[80, 85], [90, 95], [100, 105]])
x1 = x_input[:, 0].reshape((1, n_steps))
x2 = x_input[:, 1].reshape((1, n_steps))
yhat = model.predict([x1, x2], verbose=0)
print(yhat)

[[205.76938]]
